In [ ]:
'''
    Notebook: P300 Classification using DeepCNN
    Created By: Rauf Momin
    Contact: raufmomin1999@gmail.com
'''

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, glob
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
import time

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(21, 45, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding='valid'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, kernel_size=2, strides=1, padding='valid'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation = 'selu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1, activation='sigmoid')
], name = "DeepCNN")

model.compile(optimizer = tf.keras.optimizers.Adam(3e-4), loss=tf.keras.losses.BinaryCrossentropy(), metrics = tf.keras.metrics.BinaryAccuracy())
model.summary()


Model: "DeepCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 21, 45, 1)         4         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 22, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 18, 16)         416       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 6, 18, 16)         0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 18, 16)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 18, 16)         64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 9, 16)          0   

In [ ]:
base_path = '/content/P300-SSVEP-Dataset/'
clean_path = base_path + 'clean-data/'
noisy_path = base_path + 'noisy-data/'

In [ ]:
def get_data(name):
    if name is 'clean':
        path = clean_path
    else:
        path = noisy_path

    X_data = []
    Y_data = []
    subjects = sorted(os.listdir(path))

    for sub in subjects:
        file_path = f'{path}{sub}'
        mat = sio.loadmat(file_path)
        print(file_path)
        mat_data = mat['f']

        if name is 'clean':
            X = mat_data[0][0][3]
            Y = mat_data[0][0][4]
        else:
            X = mat_data[0][0][0]
            Y = mat_data[0][0][1]
            
    
        for i in range(Y.shape[1]):
            sample_x = X[:, :, i]
            sample_y = Y[:, i][0]

            X_data.append(sample_x)
            Y_data.append(sample_y)

    return np.array(X_data), np.array(Y_data)

In [ ]:
x_data, y_data = get_data('clean')

x_data = np.expand_dims(x_data, axis = 3)
y_data = np.expand_dims(y_data, axis = 1)

print(x_data.shape, y_data.shape)

/content/P300-SSVEP-Dataset/clean-data/features_sub1.mat
/content/P300-SSVEP-Dataset/clean-data/features_sub2.mat
/content/P300-SSVEP-Dataset/clean-data/features_sub3.mat
/content/P300-SSVEP-Dataset/clean-data/features_sub4.mat
/content/P300-SSVEP-Dataset/clean-data/features_sub5.mat
/content/P300-SSVEP-Dataset/clean-data/features_sub6.mat
(2700, 21, 45, 1) (2700, 1)


In [ ]:
# Clean
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for fold_num, (train_index, test_index) in enumerate(kfold.split(x_data, y_data)):
        
    print('===========>', f'FOLD {fold_num}', '<===========')
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    print(np.array(x_train).shape, np.array(y_train).shape, np.array(x_test).shape, np.array(y_test).shape)

    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(21, 45, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding='valid'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, kernel_size=2, strides=1, padding='valid'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200, activation = 'selu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name = "DeepCNN")

    model.compile(optimizer = tf.keras.optimizers.Adam(3e-4), loss=tf.keras.losses.BinaryCrossentropy(), metrics = tf.keras.metrics.BinaryAccuracy())
    # model.summary()

    start_time = time.process_time()
    history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, verbose = False)
    print('Training Time: ', time.process_time() - start_time, 'seconds')
    
    print('Train evaluate')
    model.evaluate(x_train, y_train)
    print('Test evaluate')
    model.evaluate(x_test, y_test)

    model.save(f'/content/model_{fold_num}.h5')

===========> FOLD 0 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  61.04443348099999 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1063 - binary_accuracy: 0.9593
Test evaluate
17/17 [==============================] - 0s 4ms/step - loss: 0.3414 - binary_accuracy: 0.8981
===========> FOLD 1 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  61.060412377999995 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1284 - binary_accuracy: 0.9426
Test evaluate
17/17 [==============================] - 0s 4ms/step - loss: 0.2305 - binary_accuracy: 0.9056
===========> FOLD 2 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  61.08328956899999 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1136 - binary_accuracy: 0.9588
Test evaluate
17/17 [==============================] - 0s 4ms/step - 

In [ ]:
x_data, y_data = get_data('noisy')

x_data = np.expand_dims(x_data, axis = 3)
y_data = np.expand_dims(y_data, axis = 1)

print(x_data.shape, y_data.shape)

/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub1.mat
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub2.mat
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub3.mat
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub4.mat
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub5.mat
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub6.mat
(2700, 21, 45, 1) (2700, 1)


In [ ]:
# Noisy
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for fold_num, (train_index, test_index) in enumerate(kfold.split(x_data, y_data)):
        
    print('===========>', f'FOLD {fold_num}', '<===========')
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    print(np.array(x_train).shape, np.array(y_train).shape, np.array(x_test).shape, np.array(y_test).shape)

    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(21, 45, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding='valid'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, kernel_size=2, strides=1, padding='valid'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200, activation = 'selu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name = "DeepCNN")

    model.compile(optimizer = tf.keras.optimizers.Adam(3e-4), loss=tf.keras.losses.BinaryCrossentropy(), metrics = tf.keras.metrics.BinaryAccuracy())
    # model.summary()

    start_time = time.process_time()
    history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, verbose = False)
    print('Training Time: ', time.process_time() - start_time, 'seconds')
    
    print('Train evaluate')
    model.evaluate(x_train, y_train)
    print('Test evaluate')
    model.evaluate(x_test, y_test)

    model.save(f'/content/model_noisy_{fold_num}.h5')

===========> FOLD 0 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  60.179655628000035 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1753 - binary_accuracy: 0.9366
Test evaluate
17/17 [==============================] - 0s 4ms/step - loss: 0.3548 - binary_accuracy: 0.8796
===========> FOLD 1 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  61.19765122299998 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1849 - binary_accuracy: 0.9269
Test evaluate
17/17 [==============================] - 0s 4ms/step - loss: 0.3318 - binary_accuracy: 0.8852
===========> FOLD 2 <===========
(2160, 21, 45, 1) (2160, 1) (540, 21, 45, 1) (540, 1)
Training Time:  61.11929445999999 seconds
Train evaluate
68/68 [==============================] - 0s 4ms/step - loss: 0.1825 - binary_accuracy: 0.9264
Test evaluate
17/17 [==============================] - 0s 4ms/step - 

In [ ]:
def get_data(name, subject_index):
    if name is 'clean':
        path = clean_path
    else:
        path = noisy_path

    X_data = []
    Y_data = []
    subjects = sorted(os.listdir(path))
    sub = subjects[subject_index]
    file_path = f'{path}{sub}'
    mat = sio.loadmat(file_path)
    print(file_path)
    mat_data = mat['f']

    if name is 'clean':
        X = mat_data[0][0][3]
        Y = mat_data[0][0][4]
    else:
        X = mat_data[0][0][0]
        Y = mat_data[0][0][1]

    for i in range(Y.shape[1]):
        sample_x = X[:, :, i]
        sample_y = Y[:, i][0]

        X_data.append(sample_x)
        Y_data.append(sample_y)

    return np.expand_dims(np.array(X_data), axis = 3), np.expand_dims(np.array(Y_data), axis = 1)

In [ ]:
for i in range(6):
    preds = []
    print(f'\nPredicting subject Clean {i + 1}')
    x, y = get_data('clean', i)

    models = ['/content/model_0.h5', '/content/model_1.h5', '/content/model_2.h5', '/content/model_3.h5', '/content/model_4.h5',]

    for model_path in models:
        print(f'Predicting {model_path}')
        model = tf.keras.models.load_model(model_path)
        pred = model.predict(x).squeeze()
        preds.append(pred)
        
    preds = np.array(preds)
    sub_df = pd.DataFrame(np.round(np.average(preds, axis = 0)).astype(np.int16))
    sub_df.to_csv(f'/content/preds_clean_subject_{i + 1}.csv', index = False)


Predicting subject Clean 1
/content/P300-SSVEP-Dataset/clean-data/features_sub1.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Clean 2
/content/P300-SSVEP-Dataset/clean-data/features_sub2.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Clean 3
/content/P300-SSVEP-Dataset/clean-data/features_sub3.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Clean 4
/content/P300-SSVEP-Dataset/clean-data/features_sub4.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Clean 5
/content/P30

In [ ]:
for i in range(6):
    preds = []
    print(f'\nPredicting subject Noisy {i + 1}')
    x, y = get_data('noisy', i)

    models = ['/content/model_0.h5', '/content/model_1.h5', '/content/model_2.h5', '/content/model_3.h5', '/content/model_4.h5',]

    for model_path in models:
        print(f'Predicting {model_path}')
        model = tf.keras.models.load_model(model_path)
        pred = model.predict(x).squeeze()
        preds.append(pred)

    preds = np.array(preds)
    sub_df = pd.DataFrame(np.round(np.average(preds, axis = 0)).astype(np.int16))
    sub_df.to_csv(f'/content/preds_noisy_subject_{i + 1}.csv', index = False)


Predicting subject Noisy 1
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub1.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Noisy 2
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub2.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Noisy 3
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub3.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject Noisy 4
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub4.mat
Predicting /content/model_0.h5
Predicting /content/model_1.h5
Predicting /content/model_2.h5
Predicting /content/model_3.h5
Predicting /content/model_4.h5

Predicting subject 

In [ ]:
for i in range(6):
    print(f'\nEvaluating subject Clean {i + 1}')
    x, y = get_data('clean', i)

    models = ['/content/model_0.h5', '/content/model_1.h5', '/content/model_2.h5', '/content/model_3.h5', '/content/model_4.h5',]

    for model_path in models:
        print(f'Evaluating {model_path}')
        model = tf.keras.models.load_model(model_path)
        model.evaluate(x, y)


Evaluating subject Clean 1
/content/P300-SSVEP-Dataset/clean-data/features_sub1.mat
Evaluating /content/model_0.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.2242 - binary_accuracy: 0.9200
Evaluating /content/model_1.h5
15/15 [==============================] - 0s 5ms/step - loss: 0.2074 - binary_accuracy: 0.9111
Evaluating /content/model_2.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.2195 - binary_accuracy: 0.9289
Evaluating /content/model_3.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.1988 - binary_accuracy: 0.9311
Evaluating /content/model_4.h5
15/15 [==============================] - 0s 5ms/step - loss: 0.1990 - binary_accuracy: 0.9289

Evaluating subject Clean 2
/content/P300-SSVEP-Dataset/clean-data/features_sub2.mat
Evaluating /content/model_0.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.1782 - binary_accuracy: 0.9422
Evaluating /content/model_1.h5
15/15 [==============================] - 0s 3ms/s

In [ ]:
for i in range(6):
    print(f'\nEvaluating subject Noisy {i + 1}')
    x, y = get_data('noisy', i)

    models = ['/content/model_noisy_0.h5', '/content/model_noisy_1.h5', '/content/model_noisy_2.h5', '/content/model_noisy_3.h5', '/content/model_noisy_4.h5',]

    for model_path in models:
        print(f'Evaluating {model_path}')
        model = tf.keras.models.load_model(model_path)
        model.evaluate(x, y)


Evaluating subject Noisy 1
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub1.mat
Evaluating /content/model_noisy_0.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.2376 - binary_accuracy: 0.9156
Evaluating /content/model_noisy_1.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.2509 - binary_accuracy: 0.9044
Evaluating /content/model_noisy_2.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.2600 - binary_accuracy: 0.9067
Evaluating /content/model_noisy_3.h5
15/15 [==============================] - 0s 3ms/step - loss: 0.2513 - binary_accuracy: 0.9133
Evaluating /content/model_noisy_4.h5
15/15 [==============================] - 0s 5ms/step - loss: 0.2790 - binary_accuracy: 0.8911

Evaluating subject Noisy 2
/content/P300-SSVEP-Dataset/noisy-data/noisyfeatures_sub2.mat
Evaluating /content/model_noisy_0.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.1645 - binary_accuracy: 0.9400
Evaluating /content/model_noisy_1.